In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
ipl2020=pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\IPL Ball-by-Ball 2008-2020\IPL Ball-by-Ball 2008-2020.csv')
ipl2021=pd.read_csv(r'C:\Users\Yogesh Arora\Downloads\archive\IPL Ball-by-Ball Dataset 2021.csv')

In [3]:
ipl2020.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [4]:
ipl2021.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,1254058,1,0,1,RG Sharma,CA Lynn,Mohammed Siraj,2,0,2,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
1,1254058,1,0,2,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
2,1254058,1,0,3,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
3,1254058,1,0,4,RG Sharma,CA Lynn,Mohammed Siraj,2,0,2,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore
4,1254058,1,0,5,RG Sharma,CA Lynn,Mohammed Siraj,0,0,0,0,0,NaN,NaN,NaN,NaN,Mumbai Indians,Royal Challengers Bangalore


In [5]:
matchdata = pd.concat(
    map(pd.read_csv, [r'C:\Users\Yogesh Arora\Downloads\IPL Ball-by-Ball 2008-2020\IPL Ball-by-Ball 2008-2020.csv', r'C:\Users\Yogesh Arora\Downloads\archive\IPL Ball-by-Ball Dataset 2021.csv']), ignore_index=True)

In [6]:
matchdata.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [7]:
matchdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198569 entries, 0 to 198568
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                198569 non-null  int64 
 1   inning            198569 non-null  int64 
 2   over              198569 non-null  int64 
 3   ball              198569 non-null  int64 
 4   batsman           198569 non-null  object
 5   non_striker       198569 non-null  object
 6   bowler            198569 non-null  object
 7   batsman_runs      198569 non-null  int64 
 8   extra_runs        198569 non-null  int64 
 9   total_runs        198569 non-null  int64 
 10  non_boundary      198569 non-null  int64 
 11  is_wicket         198569 non-null  int64 
 12  dismissal_kind    9764 non-null    object
 13  player_dismissed  9764 non-null    object
 14  fielder           6968 non-null    object
 15  extras_type       10500 non-null   object
 16  batting_team      198569 non-null  obj

In [8]:
        batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
        batsmen_scores_30 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_30"})
        batsmen_scores_50 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_50"})
        batsmen_scores_100 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_100"})
        batsmen_scores_0 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
            rename(columns={"batsman_runs": "total_runs_0"})
        batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
            rename(columns={"over": "total_balls_faced"})
        batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_faced"})
        batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_6"})
        batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
            rename(columns={"batsman_runs": "run_4"})
        batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())
       
        
        df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position,batsmen_scores_30,batsmen_scores_50,batsmen_scores_100,batsmen_scores_0]
        batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                                   drop_duplicates(), on=['id', 'batsman'], how='left')
        batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'Playing Team',
                                        'bowling_team': 'Opponent Team'}, inplace=True)
       

In [49]:
batsmen_summary

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,total_runs_100,total_runs_0,batsmen_innings,Playing Team,Opponent Team
0,335982,AA Noffke,9,12,11.0,NaN,1.0,0,0,0,0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
1,335982,B Akhil,0,2,2.0,NaN,NaN,0,0,0,0,1,2,Royal Challengers Bangalore,Kolkata Knight Riders
2,335982,BB McCullum,158,77,73.0,13.0,10.0,0,1,1,1,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,CL White,6,10,8.0,NaN,NaN,0,0,0,0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders
4,335982,DJ Hussey,12,12,12.0,NaN,1.0,0,0,0,0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12486,1254078,N Rana,0,1,1.0,NaN,NaN,1,0,0,0,1,2,Kolkata Knight Riders,Punjab Kings
12487,1254078,RA Tripathi,41,32,32.0,NaN,7.0,0,1,0,0,0,2,Kolkata Knight Riders,Punjab Kings
12488,1254078,Ravi Bishnoi,1,5,4.0,NaN,NaN,0,0,0,0,0,1,Punjab Kings,Kolkata Knight Riders
12489,1254078,SP Narine,0,5,4.0,NaN,NaN,0,0,0,0,1,2,Kolkata Knight Riders,Punjab Kings


In [77]:
        bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "total_wickets"})
        bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
            rename(columns={"over": "total_balls_bowled"})
        bowler_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'bowler'])['ball'].count()).\
            rename(columns={"ball": "total_legal_balls_bowled"})
        # calculating the number of maiden overs bowled in a match
        bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs_given"})
        #matchdata_runs_per_over = pd.DataFrame(self.matchdata.groupby(['matchid', 'bowlername', 'over_num'])['batsmanscorevalue'].sum()).reset_index()
        #matchdata_runs_per_over['maiden_overs'] = np.where(matchdata_runs_per_over['batsmanscorevalue'] == 0, 1, 0)
        #matchdata_runs_per_over = matchdata_runs_per_over.groupby(['matchid', 'bowlername'])['maiden_overs'].sum()
        # combining all the features created so far
        bowler_wickets_3 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==3).astype(int).\
            rename(columns={"is_wicket": "total_wickets_3"})
        bowler_wickets_4 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==4).astype(int).\
            rename(columns={"is_wicket": "total_wickets_4"})
        bowler_wickets_5 = pd.DataFrame((matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count())==5).astype(int).\
            rename(columns={"is_wicket": "total_wickets_5"})
        bowler_maiden = pd.DataFrame(matchdata.groupby(['id', 'bowler', 'over'])['total_runs'].sum()).reset_index()
        bowler_maiden['maiden_overs'] = np.where(bowler_maiden['total_runs'] == 0, 1, 0)
        bowler_maiden = bowler_maiden.groupby(['id', 'bowler'])['maiden_overs'].sum()
        
        #legbyes to be considered
        
        df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given,bowler_wickets_3,bowler_wickets_4,bowler_wickets_5,bowler_maiden]
        bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
        # adding additional columns just in case we need for modeling
        bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
        bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'Playing team',
                                       'bowling_team': 'Opponent Team'}, inplace=True)
        bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']
        bowler_summary['eco less than 5'] = (bowler_summary['economy_rate']<=5).astype(int)
        bowler_summary['eco 5-5.99'] = ((bowler_summary['economy_rate']<6)&(bowler_summary['economy_rate']>=5)).astype(int)
        bowler_summary['eco 6-7'] = ((bowler_summary['economy_rate']>=6)&(bowler_summary['economy_rate']<7)).astype(int)
        bowler_summary['eco 7-10'] = ((bowler_summary['economy_rate']>=7)&(bowler_summary['economy_rate']<10)).astype(int)
        bowler_summary['eco 11-12'] = ((bowler_summary['economy_rate']>=11)&(bowler_summary['economy_rate']<12)).astype(int)
        bowler_summary['eco greater than 12'] = (bowler_summary['economy_rate']>12).astype(int)
        
        
        

In [79]:
bowler_summary

,id,playername,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,total_wickets_3,total_wickets_4,total_wickets_5,maiden_overs,bowlers_innings,Playing team,Opponent Team,economy_rate,eco less than 5,eco 5-5.99,eco 6-7,eco 7-10,eco 11-12,eco greater than 12
0,335982,AA Noffke,1.0,25,24,35,0.0,0.0,0.0,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,8.750000,0,0,0,1,0,0
1,335982,AB Agarkar,3.0,28,24,21,1.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,5.250000,0,1,0,0,0,0
2,335982,AB Dinda,2.0,20,18,7,0.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2.333333,1,0,0,0,0,0
3,335982,CL White,NaN,7,6,22,NaN,NaN,NaN,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,22.000000,0,0,0,0,0,1
4,335982,I Sharma,1.0,19,18,6,0.0,0.0,0.0,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2.000000,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9914,1254078,Mohammed Shami,1.0,25,24,24,0.0,0.0,0.0,0,2,Kolkata Knight Riders,Punjab Kings,6.000000,0,0,1,0,0,0
9915,1254078,PJ Cummins,2.0,19,18,30,0.0,0.0,0.0,0,1,Punjab Kings,Kolkata Knight Riders,10.000000,0,0,0,0,0,0
9916,1254078,Ravi Bishnoi,NaN,24,24,19,NaN,NaN,NaN,0,2,Kolkata Knight Riders,Punjab Kings,4.750000,1,0,0,0,0,0
9917,1254078,SP Narine,2.0,24,24,22,0.0,0.0,0.0,0,1,Punjab Kings,Kolkata Knight Riders,5.500000,0,1,0,0,0,0


bowler_runs_given

In [75]:
matchdata_runs_per_over = pd.DataFrame(matchdata.groupby(['id', 'bowler', 'over'])['batsman_runs'].sum()).reset_index()
matchdata_runs_per_over['maiden_overs'] = np.where(matchdata_runs_per_over['batsman_runs'] == 0, 1, 0)
matchdata_runs_per_over = matchdata_runs_per_over.groupby(['id', 'bowler'])['maiden_overs'].sum()

In [76]:
matchdata_runs_per_over

id       bowler        
335982   AA Noffke         0
         AB Agarkar        0
         AB Dinda          0
         CL White          0
         I Sharma          0
                          ..
1254078  Mohammed Shami    0
         PJ Cummins        0
         Ravi Bishnoi      0
         SP Narine         0
         Shivam Mavi       0
Name: maiden_overs, Length: 9919, dtype: int32

In [63]:
bowler_runs_given.loc[bowler_runs_given['id'] == 335986]

KeyError: 'id'

In [42]:
  ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='inner')

In [50]:
        MINAVGBALLSFACED = 8
        MINAVGBOWLSBOWLED = 6
        player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
        player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
        conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
                      (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
        choices = ['AllRounder', 'Bowler']
        player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
        player_avg = player_avg.reset_index()

In [46]:
 ipl_merged_scorecard.isnull().sum()

id                             0
playername                     0
total_runs                     0
total_balls_faced              0
total_legal_balls_faced       44
run_6                       3337
run_4                       2593
is_wicket                      0
total_runs_30                  0
total_runs_50                  0
total_runs_100                 0
total_runs_0                   0
batsmen_innings                0
Playing Team                   0
Opponent Team_x                0
total_wickets               2428
total_balls_bowled             0
total_legal_balls_bowled       0
total_runs_given               0
total_wickets_3             2428
total_wickets_4             2428
total_wickets_5             2428
bowlers_innings                0
Playing team                   0
Opponent Team_y                0
economy_rate                   0
eco less than 5                0
eco 5-5.99                     0
eco 6-7                        0
eco 7-10                       0
eco 11-12 

In [51]:
player_avg

,playername,total_balls_faced,total_balls_bowled,playing_role
0,A Ashish Reddy,8.066667,12.266667,AllRounder
1,A Chandila,3.500000,24.000000,Bowler
2,A Choudhary,6.666667,22.000000,Bowler
3,A Dananjaya,5.000000,25.000000,Bowler
4,A Flintoff,19.000000,22.000000,AllRounder
...,...,...,...,...
374,YA Abdulla,1.000000,24.000000,Bowler
375,YK Pathan,14.702703,14.567568,AllRounder
376,YS Chahal,5.000000,21.400000,Bowler
377,Yuvraj Singh,19.414286,12.228571,AllRounder


In [ ]:
fielder_data = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (matchdata['dismissal_kind'].isin(['caught','caught and bowled','run out','stumped'])))].
                                      groupby(['fielder','dismissal_kind'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "Fielding_stats"})

In [ ]:
fielder_data

In [48]:
ipl_merged_scorecard.head()

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,bowlers_innings,Playing team,Opponent Team_y,economy_rate,eco less than 5,eco 5-5.99,eco 6-7,eco 7-10,eco 11-12,eco greater than 12
0,335982,AA Noffke,9,12,11.0,NaN,1.0,0,0,0,...,1,Kolkata Knight Riders,Royal Challengers Bangalore,8.750000,0,0,0,1,0,0
1,335982,CL White,6,10,8.0,NaN,NaN,0,0,0,...,1,Kolkata Knight Riders,Royal Challengers Bangalore,22.000000,0,0,0,0,0,1
2,335982,JH Kallis,8,7,7.0,1.0,NaN,0,0,0,...,1,Kolkata Knight Riders,Royal Challengers Bangalore,11.750000,0,0,0,0,1,0
3,335982,P Kumar,18,17,16.0,2.0,1.0,0,0,0,...,1,Kolkata Knight Riders,Royal Challengers Bangalore,9.250000,0,0,0,1,0,0
4,335982,SB Joshi,3,8,8.0,NaN,NaN,0,0,0,...,1,Kolkata Knight Riders,Royal Challengers Bangalore,8.666667,0,0,0,1,0,0


In [ ]:
  batsmen_score = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
            rename(columns={"batsman_runs": "total_runs"})
    
   